In [2]:
from youtube_transcript_api import YouTubeTranscriptApi
import re
from heapq import nlargest

First we'll get the transcript using youtube transcript api

In [5]:
ytt_api = YouTubeTranscriptApi()
fetched_transcript = ytt_api.fetch('ExvEIOWB-H0')

In [7]:
transcript = ''

We get like several snippet objects with text and timeframes and we need to extract the text alone

In [14]:
for snippet in fetched_transcript:
    transcript += snippet.text

In [16]:
snippet_count = len(fetched_transcript)
snippet_count

345

In [18]:
transcript

"793 in Lindisfarne monastery here is alittle bit on fire it was set on fire bya group of raiders commonly known asVikings who saw the wealthy poorlydefended monastery is easy pickingswhich to be fair it was the raid onLindisfarne is often considered to bethe beginning of the period of Europeanhistory known as the Viking Age becausehere after the number of Seaborn raidsand the amount of Viking activity acrossEurope increased dramatically so whowere these Vikingswell they originated from Scandinaviawhat is now Denmark Norway and Sweden beaware that these weren't a single groupof people they had different social andreligious practices and were more thanhappy to raid each other when it wasprofitable speaking of raiding Vikinghas become synonymous with raider butit's important to note that many Vikingswere simply seaborne trade is a moststayed at home farming farming inScandinavia was a mixed bag and in thenorth of the Viking territories it waspractically impossible because coldsouthern de

In [20]:
from nltk import pos_tag
from nltk.corpus import wordnet
from nltk.corpus import words
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize, sent_tokenize
import nltk
import string
import re

nltk.download('words')

[nltk_data] Downloading package words to
[nltk_data]     C:\Users\preme\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!


True

In [21]:
final_transcript = ''


clean_transcript = transcript.replace('\xa0', ' ').replace('\n', ' ').strip() # some transcripts are literred with \xa0s and unnecessary new lines
clean_transcript = re.sub(r'\s+', ' ', clean_transcript)  # Replace multiple spaces/newlines with single space
clean_transcript = re.sub(r'[^\w\s\.\?!]', '', clean_transcript)  # Remove everything except words and sentence enders

In [24]:
clean_transcript

'793 in Lindisfarne monastery here is alittle bit on fire it was set on fire bya group of raiders commonly known asVikings who saw the wealthy poorlydefended monastery is easy pickingswhich to be fair it was the raid onLindisfarne is often considered to bethe beginning of the period of Europeanhistory known as the Viking Age becausehere after the number of Seaborn raidsand the amount of Viking activity acrossEurope increased dramatically so whowere these Vikingswell they originated from Scandinaviawhat is now Denmark Norway and Sweden beaware that these werent a single groupof people they had different social andreligious practices and were more thanhappy to raid each other when it wasprofitable speaking of raiding Vikinghas become synonymous with raider butits important to note that many Vikingswere simply seaborne trade is a moststayed at home farming farming inScandinavia was a mixed bag and in thenorth of the Viking territories it waspractically impossible because coldsouthern denm

In [28]:
from deepmultilingualpunctuation import PunctuationModel

# Load model
model = PunctuationModel()


# Restore punctuation & capitalization
punctuated_text = model.restore_punctuation(clean_transcript)

print(punctuated_text)


Device set to use cpu
C:\Users\preme\anaconda3\Lib\site-packages\transformers\pipelines\token_classification.py:170: UserWarning: `grouped_entities` is deprecated and will be removed in version v5.0.0, defaulted to `aggregation_strategy="AggregationStrategy.NONE"` instead.
  warnings.warn(


793 in Lindisfarne monastery. here is alittle bit on fire. it was set on fire bya group of raiders, commonly known asVikings, who saw the wealthy, poorlydefended monastery is easy pickingswhich, to be fair, it was. the raid onLindisfarne is often considered to bethe beginning of the period of Europeanhistory known as the Viking Age, becausehere after the number of Seaborn raidsand the amount of Viking activity acrossEurope increased dramatically. so whowere these Vikingswell, they originated from Scandinaviawhat is now Denmark, Norway and Sweden beaware that these werent a single groupof people. they had different social andreligious practices and were more thanhappy to raid each other when it wasprofitable. speaking of raiding, Vikinghas become synonymous with raider, butits important to note that many Vikingswere simply seaborne. trade is a moststayed at home farming. farming inScandinavia was a mixed bag, and in thenorth of the Viking territories it waspractically impossible, becaus

We'll make a list of the lemmatized words in the transcript and add them to words_transcript list

In [29]:
sentences = sent_tokenize(punctuated_text)
wnl = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))
words_transcript = []
for sentence in sentences:
    for word in nltk.word_tokenize(sentence):
        if word not in stop_words and word.isalpha():
            words_transcript.append(wnl.lemmatize(word))

In [35]:
word_freq = nltk.FreqDist(words_transcript) # creates a frequency distribution for this particular list

In [37]:
word_freq

FreqDist({'one': 10, 'Vikings': 10, 'Denmark': 8, 'many': 8, 'made': 8, 'known': 7, 'Viking': 7, 'people': 7, 'would': 7, 'Norway': 6, ...})

In [39]:
 top_words = [word[0] for word in word_freq.most_common(10)] # get the top 10 words

In [41]:
top_words

['one',
 'Vikings',
 'Denmark',
 'many',
 'made',
 'known',
 'Viking',
 'people',
 'would',
 'Norway']

Here we are defining a custom sentence score for each sentence if it contains one of our top words, so basically the sentences with the most number of top words will have the highest sentence score and will be displayed as our summmary

In [44]:
summary = []
for sentence in sentences:
    sentence_words = nltk.word_tokenize(sentence.lower())
    sentence_score = 0
    for word in sentence_words:
        if wnl.lemmatize(word) in top_words:
            sentence_score += 1
    summary.append((sentence, sentence_score))
    
    

In [59]:
generated_summary = ''

We are choosing the 3 sentences with the largest sentence score and displaying them

In [62]:
for sentence in nlargest(3, summary, key=lambda x:x[1]): # ranking according to score
    generated_summary += sentence[0]
    print(sentence[0])

by the mid 10 centuriesmmark was still divided into manykingdoms, and it was this one, the kingdomof Jutland, which was ruled by a mancalled gone, the old Gorn being old, diedin 958 and was succeeded by his sonHarald, better known as Harald BluetoothHarald would go on to conquer theentirety of the now gormless Denmarkwhich is one of the reasons hes sofamous.
that is untila certain King, Harald fairhair, shows uparound 872, but again, no one knows forsureit was he who established a power baseover the course of a few decades, gainedover lordship over the north, and thendefeated the confederation of pettykings at the Battle of half his fieldand it was after his victory that Haralddeclared himself to be the first king ofthe Norwegians in what is now Denmarkthe ninth century isnt much clearer, andactually the name Denmark wasnt useduntil about the Year 930it is known, though, from the records ofthe Frankish Empire of Charlemagne, thatsome of the lords who lived there willdo the great deal o

Ok so let us now generate a summary using bart to compare the both of them and evaluate our own summary generated by nlp techniques

In [65]:
generated_summary

'by the mid 10 centuriesmmark was still divided into manykingdoms, and it was this one, the kingdomof Jutland, which was ruled by a mancalled gone, the old Gorn being old, diedin 958 and was succeeded by his sonHarald, better known as Harald BluetoothHarald would go on to conquer theentirety of the now gormless Denmarkwhich is one of the reasons hes sofamous.that is untila certain King, Harald fairhair, shows uparound 872, but again, no one knows forsureit was he who established a power baseover the course of a few decades, gainedover lordship over the north, and thendefeated the confederation of pettykings at the Battle of half his fieldand it was after his victory that Haralddeclared himself to be the first king ofthe Norwegians in what is now Denmarkthe ninth century isnt much clearer, andactually the name Denmark wasnt useduntil about the Year 930it is known, though, from the records ofthe Frankish Empire of Charlemagne, thatsome of the lords who lived there willdo the great deal o

This was summarization done using classic nlp techniques which are a bit crude so now we will use a transformer(bart) to summarize

In [68]:
from transformers import pipeline

# Load summarization pipeline with pretrained BART
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

Device set to use cpu


We need to split our text into chunks as maximum token length of bart that is allowed is 1024 so we split it into chunks and also add an overlap of 2 sentences to pertain context between chunks

In [71]:
from transformers import BartTokenizer

tokenizer = BartTokenizer.from_pretrained("facebook/bart-large-cnn")

overlap = 2
chunks = []
current_chunk = []
current_length = 0
chunk_size = 900  # in tokens, not words

for sentence in sentences:
    sentence_len = len(tokenizer.encode(sentence))
    if current_length + sentence_len <= chunk_size:
        current_chunk.append(sentence)
        current_length += sentence_len
    else:
        chunks.append(" ".join(current_chunk))

        # Keep last few sentences for overlap
        current_chunk = current_chunk[-overlap:] + [sentence]
        current_length = sum(len(tokenizer.encode(s)) for s in current_chunk)

if current_chunk:
    chunks.append(" ".join(current_chunk))

# Check actual token count per chunk
for i, chunk in enumerate(chunks):
    print(f"Chunk {i+1}: {len(tokenizer.encode(chunk))} tokens")


Chunk 1: 802 tokens
Chunk 2: 850 tokens
Chunk 3: 838 tokens
Chunk 4: 575 tokens


In [73]:
chunk_summaries = []
for chunk in chunks:
    bart_summary = summarizer(chunk, max_length=300, min_length=50, do_sample=False) # summarize all the chunks together
    chunk_summaries.append(bart_summary[0]['summary_text'])


In [74]:
bart_summary

[{'summary_text': 'The Viking Age is considered by many to mark the end ofthe Viking Age. The reason for this islargely because theKing Nations were part of the Christianworld and were no longer able to raidwith impunity, or even defeat theirenemies. Denmark, Sweden and Norway wereChristian kingdoms just like any otherand so there were rules. you couldntsimply take a kingdom now just becauseyou wanted it. they changed language and thepolitics of much of Western Europe.'}]

In [77]:
combined_summary_input = " ".join(chunk_summaries)
final_summary = summarizer(combined_summary_input, max_length=250, min_length=100, do_sample=False)
print(final_summary[0]['summary_text'])


Vikings originated from Scandinaviawhat is now Denmark, Norway and Sweden. The Vikings conquered most of the northernislands across the British Isles and also settle places like Dublin and Cork. The religion of the Vikings was a form of Germanicpaganism. Harald Bluetooth was the king of Denmark from 958 to 986. He converted to Christianity after losing to the Holy Roman Empire in 960. His son, Eric, the firstbetter, went off to rule Dublin in Northumbria. Eric's sonLeif Ericson would then in the year 1001discover North America.


In [68]:
generated_summary

' by the mid 10 centuriesmmark was still divided into manykingdoms, and it was this one, the kingdomof Jutland, which was ruled by a mancalled gone, the old Gorn being old, diedin 958 and was succeeded by his sonHarald, better known as Harald BluetoothHarald would go on to conquer theentirety of the now gormless Denmarkwhich is one of the reasons hes sofamous.that is untila certain King, Harald fairhair, shows uparound 872, but again, no one knows forsureit was he who established a power baseover the course of a few decades, gainedover lordship over the north, and thendefeated the confederation of pettykings at the Battle of half his fieldand it was after his victory that Haralddeclared himself to be the first king ofthe Norwegians in what is now Denmarkthe ninth century isnt much clearer, andactually the name Denmark wasnt useduntil about the Year 930it is known, though, from the records ofthe Frankish Empire of Charlemagne, thatsome of the lords who lived there willdo the great deal 

We'll evaluate our nlp summary by comparing it with the transformer one

In [80]:
from rouge_score import rouge_scorer

scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
scores = scorer.score(final_summary[0]['summary_text'], generated_summary)

print(scores)


{'rouge1': Score(precision=0.19148936170212766, recall=0.5172413793103449, fmeasure=0.2795031055900621), 'rouge2': Score(precision=0.02564102564102564, recall=0.06976744186046512, fmeasure=0.0375), 'rougeL': Score(precision=0.09787234042553192, recall=0.26436781609195403, fmeasure=0.14285714285714285)}


In [84]:
final_summary

[{'summary_text': "Vikings originated from Scandinaviawhat is now Denmark, Norway and Sweden. The Vikings conquered most of the northernislands across the British Isles and also settle places like Dublin and Cork. The religion of the Vikings was a form of Germanicpaganism. Harald Bluetooth was the king of Denmark from 958 to 986. He converted to Christianity after losing to the Holy Roman Empire in 960. His son, Eric, the firstbetter, went off to rule Dublin in Northumbria. Eric's sonLeif Ericson would then in the year 1001discover North America."}]

All the mashed together words are the fault of the youtube transcript api